In [1]:
import os
os.chdir("C:\\Users\\Luca\\Desktop\\BICOCCA\\public\\Mercorio\\progetto\\Files")

%run -i "libraries.py"
%run -i "functions.py"

C:\Users\Luca\Anaconda3.1\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
import nltk
#nltk.download('universal_tagset')

# 1. File reading

## 1.1 Reading .csv

In [3]:
os.chdir("C:\\Users\\Luca\\Desktop\\BICOCCA\\public\\Mercorio\\progetto")

In [5]:
try:
    data = pd.read_csv("NYT-merge.csv", sep=";", encoding="UTF-8")
    data.drop("Unnamed: 0", axis=1, inplace=True)
    print("Loaded comments: %s" %(len(data)))
    
except FileNotFoundError:
    print("Merge file not found, loading every .csv")
    
    articles_path = "C:\\Users\\Luca\\Desktop\\BICOCCA\\public\\Mercorio\\progetto\\Articles\\*.csv"
    comments_path = "C:\\Users\\Luca\\Desktop\\BICOCCA\\public\\Mercorio\\progetto\\Comments\\*.csv"
    articles_list = glob.glob(articles_path)
    comments_list = glob.glob(comments_path)

    
    # Loading data
    articles = get_articles(articles_path, articles_list)
    comments = get_comments(comments_path, comments_list)

    # Merge
    data = pd.merge(articles, comments, left_on="artID", right_on="comID", how="left").drop("comID", axis=1)

    
    data = data.dropna() # around 100k rows do not match any article ID
    data = data.reset_index()
    data = data.drop("index", axis=1)
    data = data[data.Keywords != ''] # remove empty keywords
    data = data.drop_duplicates(subset=["Comments"], keep="first")
    print("Loaded comments: %s" %(len(data)))

    data.to_csv("NYT-merge.csv", sep=";", encoding="UTF-8")

Merge file not found, loading every .csv


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [04:07<00:00, 27.26s/it]


Loaded comments: 2040273


## 1.2 LDA model

In [3]:
try:
    ldamodel = gensim.models.LdaMulticore.load("model-9topic.gensim")
    print_topics=10
    topics = ldamodel.print_topics(num_words=print_topics)
    for topic in topics:
        print(topic)
        
except FileNotFoundError:
    print("Error: cannot load LDA model, file not found")

(0, '0.128*"Trump, Donald J" + 0.113*"United States Politics and Government" + 0.063*"Russia" + 0.043*"Presidential Election of 2016" + 0.037*"Federal Bureau of Investigation" + 0.029*"United States International Relations" + 0.028*"Comey, James B" + 0.026*"Cyberwarfare and Defense" + 0.023*"Russian Interference in 2016 US Elections and Ties to Trump Associates" + 0.019*"Special Prosecutors (Independent Counsel)"')
(1, '0.099*"Trump, Donald J" + 0.082*"United States Politics and Government" + 0.042*"Comey, James B" + 0.039*"Federal Bureau of Investigation" + 0.037*"Global Warming" + 0.032*"United Nations Framework Convention on Climate Change" + 0.020*"Crossword Puzzles" + 0.018*"United States International Relations" + 0.017*"Greenhouse Gas Emissions" + 0.015*"Clinton, Hillary Rodham"')
(2, '0.045*"Trump, Donald J" + 0.039*"Politics and Government" + 0.029*"United States International Relations" + 0.019*"France" + 0.018*"United States Politics and Government" + 0.017*"Macron, Emmanuel

In [4]:
data_newkey = lda_assign("NYT-9topic.csv", data, ldamodel)
#print(np.unique(data_newkey["Keywords"]))

File not found, running 'lda_to_topic' function to assign topics


100%|████████████████████████████████████████████████████████████████████| 2040273/2040273 [00:17<00:00, 119280.07it/s]


# 2. Preprocessing

In [5]:
data_prep = temp = data_newkey

text = pre_processing(temp["Comments"], stopwords=0)
data_newkey.drop("Comments", axis=1, inplace=True)
data_newkey["Comments"] = text
data_newkey.to_csv("NYT-NewKeywords.csv", sep=";", encoding="UTF-8") # Saving
print(data_newkey["Comments"].iloc[0])

text_stop = pre_processing(temp["Comments"], stopwords=1)
data_prep.drop("Comments", axis=1, inplace=True)
data_prep["Comments"] = text_stop

# Remove short comments
data_prep = remove_short(data_prep, "Comments", 9) # Remove

# Lemmatization
lemmatizer = WordNetLemmatizer()
data_prep["Comments"] = lemmatization(data_prep["Comments"])

# Remove duplicate comments
data_prep = data_prep.drop_duplicates(subset=["Comments"], keep="first")

#print(data_prep.head()); print("\n", Counter(list(data_prep["Keywords"])))


data_prep.to_csv("NYT-Preprocessing.csv", sep=";", encoding="UTF-8") # Saving

100%|█████████████████████████████████████████████████████████████████████| 2040273/2040273 [02:09<00:00, 15728.27it/s]


this project makes me happy to be a year times subscriber continue to innovate across all platforms please


100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [08:04<00:00, 164.54s/it]


In [6]:
data_newkey = pd.read_csv("NYT-NewKeywords.csv", sep=";", encoding="UTF-8")
data_newkey = data_newkey.dropna()
#np.unique(data_newkey["Keywords"])

array(['(Social)_Media', 'Environment', 'Ethics', 'Guns', 'International',
       'Laws', 'News', 'US_Elections_2016', 'US_Politics'], dtype=object)

In [7]:
%%time
X_train, X_test, y_train, y_test = train_test_split(data_newkey["Comments"], 
                                                    data_newkey["Keywords"],
                                                    stratify=data_newkey["Keywords"],
                                                    test_size=0.3, random_state=0)

count_vect = CountVectorizer()
tfidf_vect = TfidfVectorizer(min_df=10, ngram_range=(2,2))
tfidf_transformer = TfidfTransformer()
X_train_count = count_vect.fit_transform(X_train)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_count)

clf = MultinomialNB(alpha=0.25, fit_prior=False).fit(X_train_tfidf, y_train)

# Constructor for sentiment
afinn = Afinn()

# Full list of comments
com_list = list(data_newkey["Comments"])
whole_list = [word for row in com_list for word in row.split(" ")]

Wall time: 1min 24s


In [8]:
# Collecting the tags for each words
word_tags = defaultdict(Counter)
for word, pos in tqdm(brown.tagged_words(tagset='universal')):
    word_tags[word][pos] += 1
    

brown_tags_words = []
for sent in brown.tagged_sents(tagset='universal'): # Simplified version
    brown_tags_words.append(("START", "START"))
    brown_tags_words.extend([(tag[:2], word) for (word, tag) in sent])
    brown_tags_words.append(("END", "END"))

cfd_tagwords = nltk.ConditionalFreqDist(brown_tags_words)
cpd_tagwords = nltk.ConditionalProbDist(cfd_tagwords, nltk.MLEProbDist)

brown_tags = [tag for (tag, word) in brown_tags_words]
cfd_tags = nltk.ConditionalFreqDist(nltk.bigrams(brown_tags))
cpd_tags = nltk.ConditionalProbDist(cfd_tags, nltk.MLEProbDist)
    
distinct_tags = set(brown_tags)

100%|████████████████████████████████████████████████████████████████████| 1161192/1161192 [00:03<00:00, 359635.74it/s]


In [32]:
def viterbi(sentence):
    viterbi, backpointer = ([] for i in range(2))
    first_viterbi, first_backpointer = ({} for i in range(2))
    
    for tag in distinct_tags:
        if tag == "START": continue
        first_viterbi[tag] = cpd_tags["START"].prob(tag) * cpd_tagwords[tag].prob(sentence[0])
        first_backpointer[tag] = "START"
        
    viterbi.append(first_viterbi)
    backpointer.append(first_backpointer)
    currbest = max(first_viterbi.keys(), key = lambda tag: first_viterbi[tag])
    
    
    for wordindex in range(1, len(sentence)):
        this_viterbi, this_backpointer = ({} for i in range(2))
        prev_viterbi = viterbi[-1]
        
        for tag in distinct_tags:
            if tag == "START": continue
            best_previous = max(prev_viterbi.keys(),
                                key = lambda prevtag: \
            prev_viterbi[prevtag] * cpd_tags[prevtag].prob(tag) * cpd_tagwords[tag].prob(sentence[wordindex]))

            this_viterbi[tag] = prev_viterbi[best_previous] * \
            cpd_tags[ best_previous ].prob(tag) * cpd_tagwords[tag].prob(sentence[wordindex])
            this_backpointer[tag] = best_previous

        currbest = max(this_viterbi.keys(), key = lambda tag: this_viterbi[tag])
        viterbi.append(this_viterbi)
        backpointer.append(this_backpointer)
    
    
    prev_viterbi = viterbi[-1]
    best_previous = max(prev_viterbi.keys(),
                        key = lambda prevtag: prev_viterbi[prevtag] * cpd_tags[prevtag].prob("END"))

    prob_tagsequence = prev_viterbi[best_previous] * cpd_tags[best_previous].prob("END")

    # best tagsequence: we store this in reverse for now, will invert later
    best_tagsequence = ["END", best_previous]
    # invert the list of backpointers
    backpointer.reverse()

    current_best_tag = best_previous
    for bp in backpointer:
        best_tagsequence.append(bp[current_best_tag])
        current_best_tag = bp[current_best_tag]

    best_tagsequence.reverse()
    
    return(" ".join(best_tagsequence[1:-1]))

In [54]:
def category(model, sentence):
    """
    Given a sentence this function estimate the probability of belonging to one class.
    It return a dataframe with classes and estimated probabilities.
    """
    
    # https://datascienceplus.com/multi-class-text-classification-with-scikit-learn/
    if type(sentence) == list:
         sentence = " ".join(sentence)
    predictions = (model.predict_proba(count_vect.transform([sentence])))
    ref_classes = (model.classes_).tolist()
    predictions = [value for lista in predictions.tolist() for value in lista]
    
    df_prob_class =  pd.DataFrame({'Pred': predictions, 'class': ref_classes})
    df_prob_class = df_prob_class.sort_values(["Pred"], ascending=False)
    df_prob_class = df_prob_class.reset_index().drop("index", axis=1)
    
    print(tabulate(df_prob_class, headers="keys", tablefmt="psql", showindex=False))
    
    return(df_prob_class)


def random_select(length, comm_list):
    pos = np.random.choice(len(comm_list))
    sentence = (comm_list[pos:pos+length])
    sentence = [el for el in list(sentence) if len(el) > 0]
    
    temp_sent = pre_processing(sentence, stopwords=1)
    temp_sent = " ".join(temp_sent)
    
    temp_df = pd.DataFrame([temp_sent], columns=["Sentence"])
    
    temp_sent = lemmatization(temp_df["Sentence"])
    
    temp_sent = [el for el in list(temp_sent) if len(el) > 0]
    temp_sent = ' '.join(temp_sent)
    
    return(sentence, temp_sent)


def follow(topic_df, sentence, end, n_match):
    data = pd.DataFrame()
    afinn = Afinn()
    follow_freq = {}
    
    while len(follow_freq) < n_match: # I need at least a given number of following set of words
        follow_freq = {}
        string_sentence = " ".join(sentence[-end:])
        
        for comment in topic_df["Comments"]:
            if string_sentence in comment: # I only need comment with the sentence 
                comment_list = comment.split(string_sentence)

                for part in comment_list[1:]:
                    part = part.split(" ")
                    if len(part) > end: # I need at least end following words (the first element is a blank space)
                        part = part[1:]
                        following = ""

                        for i in range(end):
                            following = following + part[i] + " "
                        
                        following = following[:len(following)-1] # remove the last ""
                        
                        if following in follow_freq:
                            follow_freq[following] += 1
                        else:
                            follow_freq[following] = 1

        if len(follow_freq) < n_match:
            end -= 1
    
    follow_freq = Counter(follow_freq).most_common()
    
    # Check if null sentiment
    max_sent = 0
    for cont in range(len(follow_freq)):
        temp_follow = follow_freq[cont][0].split(" ")
        for i in range(len(temp_follow)):
            test_sent = afinn.score(temp_follow[i])
            if test_sent > max_sent:
                max_sent = test_sent
    
    if max_sent == 0:
        print("Warning! Adding word with a null sentiment")
        
    return(follow_freq, end, max_sent)  


def words_dataframe_light(topic_df, sentence, alpha, method, step, n_match, end_seq):
    if step == 0:
        end = len(sentence)
    else:
        end = end_seq
        
    status = "not ok"
    
    while status == "not ok":
        (follow_freq, end, max_sent) = follow(topic_df, sentence, end, n_match)
        
        if method == 1: # If what I care most is the sentence's meaning
            if len(follow_freq) > 0:
                status = "ok"
        if method == 0: # If what I care most is the sentence's sentiment
            if max_sent == 0:
                if end > 2:
                    end -= 1
                else:
                    print("No more words can be added for the given sentence!")
                    break
            else:
                status = "ok"
              
            
    """
    Follow freq is a tuple with "list of words", "freq".
    I create as many lists as the number of words in "list of words" which is exactly the end value
    I must have two more lists: one for store the freq count and one for the sentiment
    Words in the "list of words" which also appears at the end of the sentence are not considered
    With what I found, I create a new index value which is a combination between the freq count and the sentiment
    """        
    lists = [[] for _ in range(end+2)]
    big_list = [el for el in lists]
    
    follow_count = []
    afinn_score = []
    for cont in range(len(follow_freq)):
        temp_follow = follow_freq[cont][0].split(" ")
        sent_score = 0
        for i in range(len(temp_follow)):
            if temp_follow[i] not in sentence[-end:]: # I try not to add a word which is already at the end of the sentence
                try:
                    big_list[i].append(temp_follow[i])
                except IndexError:
                    break
                sent_score = sent_score + afinn.score(temp_follow[i])
            else:
                big_list[i].append(" ")
                sent_score += 0
                
        big_list[end].append(follow_freq[cont][1])
        big_list[end+1].append(sent_score)
        try:
            if len(big_list[end]) != len(big_list[i]): # It happens when a word in also at the end of the sentence
                del (big_list[end][-1], big_list[end+1][-1]) 
        except IndexError:
            pass

    for i in range(len(big_list[end])):
        if type(big_list[end][i]) == str:
            big_list[end][i] = 0
        if type(big_list[end+1][i]) == str:
            big_list[end+1][i] = 0

    (max_count, max_sent) = (max(big_list[end]), max(big_list[end+1]))
    count_stand = [big_list[end][cont]/max_count for cont in range(len(big_list[end]))]
    
    if max_sent == 0:
        max_sent = 0.01
    
    sent_stand = [big_list[end+1][cont]/max_sent for cont in range(len(big_list[end+1]))]
    best_pick = [(count_stand[cont]*(sent_stand[cont] + alpha)) for cont in range(len(big_list[end]))]
            
    # Creating names from the df
    variables = ["Follow" + str(i+1) for i in range(end)]
    new_vars = ["Count", "Sentiment", "Count_stand", "Sent_stand", "Best_pick"]
    for var in new_vars:
        variables.append(var)
    
    big_list.append(count_stand)
    big_list.append(sent_stand)
    big_list.append(best_pick)
    diz = {}
    for i in range(len(big_list)):
        diz[variables[i]] = big_list[i]
    

    datt = pd.DataFrame(diz, columns=diz.keys())
    
    return(datt, end, max_sent)  



def final_function_light(df, model, length, comment_list, alpha, end_seq, n_match, method):
    (sentence, prep_sentence) = random_select(length, comment_list)
    
    threshold = int(input("Type a number as the sentiment upper bound "))
    print("Sentiment threshold:", str(threshold))
    
    sentiment = 0
    step = 0
    
    # predict the class
    data_class = category(model, prep_sentence)
    print("{", sentence, "}\t predicted class:", data_class["class"].iloc[0])
    globals()["data_%s" %data_class["class"].iloc[0]] = df.loc[df["Keywords"] == data_class["class"].iloc[0]]
    
    
    while abs(sentiment) < abs(threshold):
        print("{", " ".join(sentence), "}\t sentiment score:", afinn.score(" ".join(sentence)), "\n")
        
        # find the most common following words
        (data, end, max_sent) = words_dataframe_light(globals()["data_%s" %data_class["class"].iloc[0]], 
                                                      sentence, alpha=alpha, n_match=n_match, step=step, method=method,
                                                      end_seq=end_seq)
        data = data.reset_index(drop=True)
        #data.drop("index",axis=1,inplace=True)
        
        
        if threshold > 0:
            data = data[data["Best_pick"] > 0]
            data = data.sort_values(by=['Best_pick'], ascending=False)
        else:
            data = data[data["Best_pick"] < 0]
            data = data.sort_values(by=['Best_pick'], ascending=True)
        
        
        # Tag sequence
        tag_seq_list = []
        tag_seq = ""
        for n in range(len(data)):
            for i in range(end):
                try:
                    tag_seq = tag_seq + (list(word_tags[data["Follow" + str(i+1)].iloc[n]]))[0] + " "
                except IndexError:
                    tag_seq = tag_seq + "None" + " "
            tag_seq = tag_seq[:-1]
            tag_seq_list.append(tag_seq)
        data["Tags"] = tag_seq_list
        
        best_tag_seq = viterbi(sentence[-end:])
        try:
            data_tag = data.loc[data["Tags"] == best_tag_seq]
        except TypeError:
            pass
        if len(data_tag) > 0:
            data = data_tag
        
        
        
        # Update sentence
        sentence = " ".join(sentence)
        for i in range(end):
            if data["Follow" + str(i+1)].iloc[0] != "":
                sentence = sentence + " " + data["Follow" + str(i+1)].iloc[0]
        print(sentence)
                
        sentence = sentence.split(" ")
        sentiment = afinn.score(" ".join(sentence))
        step += 1
        
    print("New sentence: {", " ".join(sentence), "}\t score:", afinn.score(" ".join(sentence)))
    return(sentence)

In [55]:
"""
df           = the dataframe with less preprocessing
model        = the best classification model
length       = number of words that should be taken to inizialize the sentence
comment_list = list of all comments from where to take the words
alpha        = smoothing parameters to create an index which combines sentiment and frequence
end_seq      = number of element at the end of the sentence that are taken as input for expanding the sentence itself
n_match      = minimum number of following sequences found before reducing the end_seq
method       = if 0 the function strictly selects part of sentence with not null sentiment
               if 1 the function may add pertinent words with null sentiment
"""

final_function_light(df=data_newkey, model=clf, length=20, comment_list=whole_list, 
                     alpha=0.05, end_seq=8, n_match=5, method=0)

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1486.64it/s]


Type a number as the sentiment upper bound 10
Sentiment threshold: 10
+-----------+-------------------+
|      Pred | class             |
|-----------+-------------------|
| 0.67452   | Laws              |
| 0.127029  | International     |
| 0.0588022 | Ethics            |
| 0.0436148 | Environment       |
| 0.0308774 | US_Politics       |
| 0.0181738 | News              |
| 0.0178023 | (Social)_Media    |
| 0.0155433 | US_Elections_2016 |
| 0.0136375 | Guns              |
+-----------+-------------------+
{ ['off', 'the', 'table', 'the', 'dreamers', 'are', 'doomed', 'they', 'have', 'no', 'other', 'card', 'to', 'play', 'other', 'than', 'the', 'threat', 'of', 'non'] }	 predicted class: Laws
{ off the table the dreamers are doomed they have no other card to play other than the threat of non }	 sentiment score: -5.0 

off the table the dreamers are doomed they have no other card to play other than the threat of non winning events
{ off the table the dreamers are doomed they have no other 

['off',
 'the',
 'table',
 'the',
 'dreamers',
 'are',
 'doomed',
 'they',
 'have',
 'no',
 'other',
 'card',
 'to',
 'play',
 'other',
 'than',
 'the',
 'threat',
 'of',
 'non',
 'winning',
 'events',
 'like',
 'the',
 'good',
 'old',
 'days',
 'save',
 'the',
 'supreme',
 'court']